In [2]:
import numpy as np
import pandas as pd

pd.options.display.max_rows = 11
pd.options.display.max_columns = 21

from nycflights13 import flights, weather, planes, airports, airlines


Es.1 Voli il giorno di Natale

In [2]:
voliNatale = flights.loc[(flights['month'] == 12) & (flights['day'] == 25)]
#voliNatale

Es.2 Voli con un ritardo in partenza o in arrivo

In [4]:
voliRitardo = flights.loc[(flights['dep_delay'] > 0) | (flights['arr_delay'] > 0) ].dropna()
#voliRitardo

Es.3 Voli con ritardo in partenza ma non in arrivo + PrimaryKey

In [3]:
flID = flights.copy()
primaryKey = np.arange(0, len(flights), 1)
flID.insert(0, "id",primaryKey)
#flID.loc[(flID['dep_delay'] > 0) & (flID['arr_delay'] < 0) ]

Es.4 Voli che hanno recuperato il ritardo

In [4]:
flCatch = flID.copy()
catchup = flCatch['dep_delay'] - flCatch['arr_delay']
flCatch.insert(10, "catchup", catchup)
flCatchDisplay = flCatch[['id', 'dep_delay', 'arr_delay', 'catchup']].copy()
#flCatchDisplay

Es.5 Numero di voli ogni giorno

In [6]:
flDays = flID.copy()
flDaysShort = flDays[['id', 'month', 'day']].copy()
flGroup = flDaysShort.groupby(['month', 'day']).size()
flGroup = flGroup.reset_index(name = 'NumeroVoli')
#flGroup

Es. 6 Giorni con più di 1000 voli

In [7]:
flBusy = flGroup.copy()
#flBusy[(flBusy['NumeroVoli'] > 1000) & (flBusy['month'] == 7)]

Es.7 Ritardo medio di partenza dei giorni di luglio con più di 1000 voli

In [26]:
flJuly = flID.copy()
filter = flBusy.copy() 
filter = filter[(filter['month'] == 7) & (filter['NumeroVoli'] > 1000)]
mask = flJuly['day'].isin(filter['day'])
flJulyF = flJuly[mask]
flJulyFMean['dep_delay'].mean().sort_values(ascending = False)

month  day
7      10     52.860702
       8      37.296646
       9      30.711499
       12     25.096154
       11     23.609392
       18     20.626789
       25     19.674134
       17     13.670707
       31      6.280843
Name: dep_delay, dtype: float64

Es.8 Join con tabella airlines

In [10]:
data = flID.copy()
flights2 = data[['year', 'month', 'day', 'hour', 'origin', 'dest', 'tailnum', 'carrier']]
flightsJoined = pd.merge(flights2, airlines, how = 'left')
#flightsJoined

Es.9 Join con planes usando solo tailnum come chiave

In [10]:
flToJoin = flights2.copy()
flPlanesJoined = pd.merge(flToJoin, planes, on = 'tailnum', how = 'left')
#flPlanesJoined.rename(columns = {"year_x" : "Anno Volo", "year_y" : "Anno Aereo"})

Es.10 Left Join Diversi

In [11]:
flToJoin = flights2.copy()
airports
flAirportDep = pd.merge(flToJoin, airports, left_on = 'origin', right_on = 'faa', how = 'left')
flAirportDest = pd.merge(flToJoin, airports, left_on = 'dest', right_on = 'faa', how = 'left')
#flAirportDep
#flAirportDest

Es.11 Destinazioni più popolari con numero di voli, ordinate decrescente

In [47]:
data = flAirportDest.copy()
dataGroup = data.groupby('name').size()
#dataGroup.sort_values(ascending = False).head(10)

Es.12 Voli effettuati da aerei Boeing

In [53]:
data = flPlanesJoined.copy()
flightsByBoeing = data.loc[data['manufacturer'] == 'BOEING']
#flightsByBoeing.rename(columns = {"year_x" : "Anno Volo", "year_y" : "Anno Aereo"})

Es.13 Voli con destinazione ad altitudine superiore a 6000 piedi

In [85]:
data = flAirportDest.copy()
flightsHigh = data.loc[data['alt'] > 6000]
#flightsHigh.sort_values(by = ['alt'], ascending = False)

Es.14 Voli con 4 motori e visibilità minore di 3 miglia

In [91]:
data = flPlanesJoined.copy()
flights4Engines = data.loc[data['engines'] == 4.0]
flights4EnginesWeather = pd.merge(flights4Engines, weather, how = 'left')
finalData = flights4EnginesWeather.loc[flights4EnginesWeather['visib'] < 3.0]
#finalData.rename(columns = {"year_x" : "Anno Volo", "year_y" : "Anno Aereo"})

Es.15 Voli con aeroporti a differenza di altitudine


In [11]:
data = flights2.copy()
altitudes = airports[['faa','alt']]
depAlt = pd.merge(data, altitudes, left_on = 'origin', right_on = 'faa', how = 'left')
finalAlt = pd.merge(depAlt, altitudes, left_on = 'dest', right_on = 'faa', how = 'left')
finalAlt = finalAlt.drop(columns = ['faa_x', 'faa_y'])
finalAlt = finalAlt.rename(columns = {'alt_x' : 'Altitudine Partenza', 'alt_y' : 'Altitudine Arrivo'})
delta = np.absolute(finalAlt['Altitudine Partenza'] - finalAlt['Altitudine Arrivo'])
finalAlt = finalAlt.assign(deltaAlt=delta)
#finalAlt.loc[finalAlt['deltaAlt'] > 6000]

Es.16 Verifica che tailnum sia davvero una chiave

Es.17 Anti Join

In [4]:
data1 = flights.copy()
data2 = airports.copy()

#anti join in due passaggi
outer = pd.merge(data1, data2, how = 'outer', left_on = 'dest', right_on = 'faa', indicator = True)
antiJoin = outer[(outer._merge=='left_only')].drop('_merge', axis = 1)
#antiJoin

Ho ottenuto tutti i voli che in dest hanno un riferimento ad un faa non presente in Airports facendo quindi un Anti-Join

In [120]:
data = antiJoin.copy()
data = data.assign(dest = pd.NA)


18926     <NA>
18927     <NA>
18928     <NA>
18929     <NA>
18930     <NA>
          ... 
334858    <NA>
334859    <NA>
334860    <NA>
334861    <NA>
334862    <NA>
Name: dest, Length: 7602, dtype: object